In [1]:
from pyspark import SparkConf, SparkContext
import re

In [2]:
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

## Demo

In [3]:
def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

In [4]:
wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)


In [5]:
result = wiki.take(1)[0]
for word in result[:50]:
    print word

Anarchism
Anarchism
is
often
defined
as
a
political
philosophy
which
holds
the
state
to
be
undesirable
unnecessary
or
harmful
The
following
sources
cite
anarchism
as
a
political
philosophy
Slevin
Carl
Anarchism
The
Concise
Oxford
Dictionary
of
Politics
Ed
Iain
McLean
and
Alistair
McMillan
Oxford
University
Press
2003
However
others
argue


## Pairs
### Intro
In this assignment you will use Spark to compute various statistics for word pairs. At the same time, you will learn some simple techniques of natural language processing.

Dataset location: /data/wiki/en_articles

Format: article_id <tab> article_text

While parsing the articles, do not forget about Unicode (even though this is an English Wikipedia dump, there are many characters from other languages), remove punctuation marks and transform words to lowercase to get the correct quantities.

### Task
Find all the pairs of two consequent words where the first word is “narodnaya”. For each pair, count the number of occurrences in the Wikipedia dump. Print all the pairs with their count in a lexicographical order. Output format is “word_pair <tab> count”, for example:

```
red_apple	100500

crazy_zoo	42
```

Note that two words in a pair are concatenated with the underscore character, and the result is in the lowercase.

One motivation for counting these continuations is to get a better understanding of the language. Some words, like “the”, have a lot of continuations, while others, like “San”, have just a few (“San Francisco”, for example). One can build a language model with these statistics. If you are interested to learn more, search for “n-gram language model” in the Internet.

In [6]:
def pairs_starting_from_word(words, first_word='word'):
    pairs = []
    
    for i, word in enumerate(words[:-1]):
        if (word == first_word):
            pair = '{}_{}'.format(word, words[i+1])
            cnt = 1
            pairs.append((pair, cnt))
        else:
            continue
    return pairs

In [9]:
# lowercase all words
wiki_lower = wiki.map(lambda words: [x.lower() for x in words])

# find pairs starting from defined word
wiki_pairs = wiki_lower.flatMap(lambda x: pairs_starting_from_word(x, 'narodnaya'))

# filtering empty elements
wiki_pairs = wiki_pairs.filter(lambda x: x != [])

# aggregate counters
wiki_red = wiki_pairs.reduceByKey(lambda a, b: a + b)

# sort values by key
wiki_red_sorted = wiki_red.sortByKey()

In [12]:
result = wiki_red_sorted.collect()
for pair, cnt in result:
    print '{}\t{}'.format(pair, cnt)

narodnaya_gazeta	1
narodnaya_volya	9


## Collocations
### Intro
PMI of two words, a & b, is defined as “PMI(a, b) = ln (P(ab) / (P(a) * P(b))”, where P(ab) is the probability of two words coming one after the other, and P(a) and P(b) are probabilities of words a & b respectively.

You will estimate probabilities with occurrence counts, that is “P(a) = # of occurrences of word a / total number of words”, and “P(ab) = # of occurrences of words ‘a b’ / total number of word pairs”.

To build an intuition behind the definition, consider the following cases:

- ***“roman empire”;*** assume that this is a unique combination, and every occurrence of “roman” is followed by “empire”, and, vice versa, every occurrence of “empire” is preceded by “roman”. In this case, “P(ab) = P(a) = P(b)”, so “PMI(a, b) = -ln P(a) = -ln P(b)”. This quantity increases when the probability of the collocation is low.

- ***“the doors”;*** let’s assume that “the” may occur with every word, independently. Thus, “P(ab) = P(a)*P(b)”, and “PMI(a, b) = ln 1 = 0”.

- ***“green idea / sleeps furiously”;*** when two words never occur together, “P(ab) = 0”, and “PMI(a, b) = -inf”. Therefore, rare combinations of coupled words have large PMI.

### Task
As for the second part of the assignment, your task is to extract collocations: that is word combinations that occur together. For example, “high school” or “roman empire”.

To find collocations, you will use NPMI (normalized pointwise mutual information) metric.

PMI of two words, a & b, is defined as “PMI(a, b) = ln (P(ab) / (P(a) * P(b))”, where P(ab) is the probability of two words coming one after the other, and P(a) and P(b) are probabilities of words a & b respectively.

You will estimate probabilities with occurrence counts, that is “P(a) = # of occurrences of word a / total number of words”, and “P(ab) = # of occurrences of words ‘a b’ / total number of word pairs”.

Therefore, rare combinations of coupled words have large PMI.

NPMI is computed as “NPMI(a, b) = PMI(a, b) / -ln P(ab)”. This normalizes the quantity to be within the range [-1; 1].

You task is a bit more complicated now:

- Extract all the words, as in the previous task.
- Filter out stopwords using the dictionary (/datasets/stop_words_en.txt ) (do not forget to convert words to the lowercase!)
- Compute all bigrams (that is, pairs of consequent words)
- Leave only bigrams with at least 500 occurrences
- Compute NPMI for every bigram (note: when computing probabilities, you need unpruned counts!)
- Sort word pairs by NPMI in the descending order
- Print top 39 word pairs, with words delimited by the underscore “_”

For example,

```
roman_empire

south_africa
```

***Hint:*** if you did everything right, “roman_empire” and “south_africa” are going to be in the result.

In [15]:
with open('/datasets/stop_words_en.txt', 'r') as f:
    stop_words = set(f.read().split())

print(len(stop_words))

319


In [18]:
stop_words

{'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amoungst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'bill',
 'both',
 'bottom',
 'but',
 'by',
 'call',
 'can',
 'cannot',
 'cant',
 'co',
 'computer',
 'con',
 'could',
 'couldnt',
 'cry',
 'de',
 'describe',
 'detail',
 'do',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eg',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'etc',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fify',
 'fill',
 'find',
 'fire',
 'first',
 'f

In [67]:
def create_bigrams(words):
    bigrams = []
    for i, word in enumerate(words[:-1]):
        pair = u'_'.join((word, words[i+1])).encode('utf-8')
        cnt = 1
        bigrams.append((pair, cnt))
    return bigrams

In [ ]:
def calc_npmi(words, pair):
    return pass

In [90]:
# total number of words pairs
tot_num_pairs = wiki_filt.flatMap(lambda words: len(words) - 1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 61.0 failed 1 times, most recent failure: Lost task 0.0 in stage 61.0 (TID 236, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 1338, in takeUpToNumLeft
    yield next(iterator)
TypeError: 'int' object is not iterable

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at or

In [84]:
# lowercase all words
wiki_lower = wiki.map(lambda words: [x.lower() for x in words])

# filter words in stop_words_en.txt
wiki_filt = wiki_lower.map(lambda words: [x for x in words if x not in stop_words])



# create bigrams
wiki_bigrams = wiki_filt.flatMap(create_bigrams)

# aggregate counters
wiki_red = wiki_bigrams.reduceByKey(lambda a, b: a + b)

# filter values by counter
wiki_red_filt = wiki_red.filter(lambda (pair, cnt): cnt >= 500)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 58.0 failed 1 times, most recent failure: Lost task 0.0 in stage 58.0 (TID 230, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/pyspark/rdd.py", line 829, in func
    initial = next(iterator)
TypeError: 'int' object is not iterable

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at o

In [80]:
result = wiki_red_filt.take(10)
for word in result:
    print word

('york_city', 625)
('north_american', 510)
('notes_references', 638)
('new_york', 4193)
('roman_catholic', 630)
('award_best', 570)
('american_singer-songwriter', 549)
('catholic_church', 582)
('world_war', 2335)
('prime_minister', 1051)
